In [4]:
import torch_xla.core.xla_model as xm
device = xm.xla_device()  # Use TPU as the device
print(f'Training on: {device}')



Training on: xla:0


NameError: name 'torch' is not defined

In [8]:
import sys
import os
from O2_Pipeline import FuncionesPipeline as fp
import importlib

# Add the path to O2_Pipeline to sys.path
sys.path.append(os.path.abspath('../../O2_Pipeline'))

importlib.reload(fp)

corpus = fp.pipeline()

ModuleNotFoundError: No module named 'O2_Pipeline'

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch_xla
import torch_xla.core.xla_model as xm
import torch_xla.distributed.parallel_loader as pl
from torch.utils.data import DataLoader, Dataset
